In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split  
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [11]:
# specifying some paths
path = r"C:\Users\janse\OneDrive\Bureaublad\Master\Data Mining Techniques\Repo\Assignment2"
testpath = os.path.join(path, 'data', 'test_set_VU_DM.csv')
trainpath = os.path.join(path, 'data', 'training_set_VU_DM.csv')
samplepath = os.path.join(path, 'data', 'train_sample.csv')
resultpath = os.path.join(path, 'data', 'result.csv')


In [12]:
# load train_main
# made a function for quick reusability during development
def reload_train():
    train_main = pd.read_csv(trainpath)
    train_main.date_time = pd.to_datetime(train_main.date_time)
    return train_main

In [13]:
train_main = reload_train()

# Evaluation

In [19]:

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

def eval_ndcg(df, srch_id_col, predict_col, value_col, n=9999999999999999):
    df = df.sort_values([srch_id_col, predict_col], ascending=False)
    k = 5
    ndcgs = []
    for i, srchid in enumerate(df[srch_id_col].unique()):
        if i == n:
            break
        if i % 10000 == 0 and i != 0:
            print(i)
            print(np.mean(ndcgs))
        r = df[df[srch_id_col] == srchid][value_col]
        ndcgs.append(ndcg_at_k(r,k))

    print(np.mean(ndcgs))


In [17]:
# select columns
cols = [x for x in train_main.columns if x.startswith('srch_')]
cols.extend([x for x in train_main.columns if x.startswith('prop_')])
cols.extend(['promotion_flag', 'random_bool', 'click_bool', 'booking_bool'])
#cols.extend(['price_usd','orig_destination_distance', 'promotion_flag'])
print(cols)

X = train_main[cols].fillna(0)
y = train_main.click_bool + 4 * train_main.booking_bool
X = X.drop(['click_bool','booking_bool'], axis=1)

['srch_id', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'promotion_flag', 'random_bool', 'click_bool', 'booking_bool']


# Nearest Neighbour

In [ ]:
from sklearn.model_selection import train_test_split  
from sklearn.neighbors import KNeighborsClassifier  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  


classifier = KNeighborsClassifier(n_neighbors=40)  
classifier.fit(X_train, y_train)  

y_pred = classifier.predict(X_test)

In [ ]:
# evaluation
results = pd.DataFrame({'srch_id': X_test.srch_id, 'predicted': y_pred, 'actual' : y_test})
eval_score = eval_ndcg(results, 'srch_id', 'predicted', 'actual')